<a href="https://colab.research.google.com/github/koyomin9zx/UITQA-Vietnamese-Question-Answering/blob/master/example_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/koyomin9zx/UITQA-Vietnamese-Question-Answering.git

In [0]:
!pip install pytorch-transformers
!pip install underthesea 
!pip install unidecode

In [0]:
!ln -s /content/drive/'My Drive'/data

In [0]:
!mv /content/UITQA-Vietnamese-Question-Answering/combine.py /content

In [0]:
# importing libraries 
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
from scipy.spatial import distance 
from collections import defaultdict, OrderedDict 
from string import punctuation
import unidecode
import re
import pandas as pd 
import numpy as np 
import glob
from combine import QA
import time


In [0]:
def load_data(path):
  data=[]
  all_files = glob.glob(path + "/*.txt")
  for file in all_files:
      passage=open(file, "r", encoding='utf-8').read()
      data.append(passage)
  return data

def vi_tokenizer(row):
    return word_tokenize(row, format="text")

def remove_accents(row):
  return unidecode.unidecode(row)

def remove_stopwords(stopwords,hl_split):
  sent = [s for s in hl_split if s not in stopwords ]
  sent = ' '.join(sent)
  return sent

def standardize_data(df,stopwords):
    hl_cleansed=[]
    remove = punctuation
    remove = remove.replace("_", "")
    pattern = "[{}]".format(remove) # create the pattern
    re_space=re.compile('\s+')
    re_trailing=re.compile('^\s+|\s+?$')
    for row in df:
        row = vi_tokenizer(row)
        row=re.sub(pattern, " ", row) 
        row=re.sub(re_space,' ',row)
        row=re.sub(re_trailing,' ',row)
        row = row.strip()
        row = remove_stopwords(stopwords,row.split())
        #row = remove_accents(row)
        row = row.lower()
        hl_cleansed.append(row)
    return hl_cleansed

## Converting 3D array of array into 1D array 
def arr_convert_1d(arr): 
    arr = np.array(arr) 
    arr = np.concatenate( arr, axis=0 ) 
    arr = np.concatenate( arr, axis=0 ) 
    return arr 
   
## Cosine Similarity 
def cosine(trans): 
    cos = [] 
    cos.append(cosine_similarity(trans[0], trans[1])) 
    return cos

def tfidf(str1, str2,tf_idf_vetor,stopwords):
    str1=standardize_data([str1],stopwords)
    str2=standardize_data([str2],stopwords)  
    corpus = [str1[0],str2[0]] 
    trans = tf_idf_vetor.transform(corpus)
    cos=cosine(trans) 
    return arr_convert_1d(cos)[0]

def relevant_ranking(query,data,vect,stopwords):
  score=defaultdict()
  i=0
  for d in data:
    t=tfidf(query, d,vect,stopwords)
    if t!=0.0:
      score[t]=d
    i+=1
  return OrderedDict(sorted(score.items(),reverse=True))

In [1]:
start = time.time()
model=QA('/content/data/BERT_Squad_WIki_UIT_pretrain') #path to model
end = time.time()
print("time load model: "+str(round((end - start),2)))

time load model: 10.14


In [0]:
data=load_data('/content/data/data_QA/data')
stopwords = set(open('/content/data/data_QA/stopwords/stopwords.txt').read().split('\n')[:-1])
vect = TfidfVectorizer(min_df=1, max_df=0.80,max_features=2000,sublinear_tf=True) 
stan_data=standardize_data(data,stopwords)
vect.fit(stan_data)

In [23]:
q="Chu Văn An quê ở đâu ?"

a=relevant_ranking(q,data,vect,stopwords)

i=0
for score,doc in a.items():
  answer = model.predict(doc,q)
  print('Question: ',q)
  print('\nAnswer: ',answer['answer'])
  print('\nScore: ',answer['confidence'])
  print('\nContent: ',doc[:200],'...')
  print('\n==========================================\n\n\n')
  i+=1
  if i==3:
    break




Question:  Chu Văn An quê ở đâu ?

Answer:  thôn Văn, Xã Quang Liệt, nay là Thanh Liệt, huyện Thanh Trì, Hà Nội

Score:  0.8909438198935057

Content:  Chu Văn An (1292 - 1370)

Chu Văn An - Nhà giáo dục đầu tiên của Việt Nam

Chu Văn An (còn gọi là Chu An) người thôn Văn, Xã Quang Liệt, nay là Thanh Liệt, huyện Thanh Trì, Hà Nội (quê mẹ). Theo Đăng  ...




Question:  Chu Văn An quê ở đâu ?

Answer:  động Hoa Lư, châu Đại Hoàng

Score:  0.6656076819135944

Content:  Đinh Bộ Lĩnh (Ất Dậu 925 – Kĩ Mão 979).

tổ nhà Đinh, ông vốn họ Đinh, tên Hoàng, Bộ Lĩnh là tước quan của sứ quân Trần Lâm phong cho, quê ở động Hoa Lư, châu Đại Hoàng (nay là huyện Gia Viễn, tỉnh Ni ...




Question:  Chu Văn An quê ở đâu ?

Answer:  1762 con ông Nguyễn Phước Luân và bà Nguyễn Thị Hoàn, cháu nội của Võ vương Nguyễn Phước Khoát.

Score:  0.5105455297919408

Content:  Gia Long Hoàng đế (1802-1819)

Ông chính tên là Nguyễn Phước Ánh, sinh nǎm 1762 con ông Nguyễn Phước Luân và bà Nguyễn Thị Hoàn, cháu nội của